In [1]:
import os

In [2]:
os.chdir("../")

In [4]:
export MLFLOW_TRACKING_URI=https://dagshub.com/jerf8010/end-to-end-mlops.mlflow
export MLFLOW_TRACKING_USERNAME=jerf8010
export MLFLOW_TRACKING_PASSWORD=e1a4af0ea32606b9c9704ac2fb3fe38cc4f13f89
python script.py

UsageError: Line magic function `%` not found.


In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/jerf8010/end-to-end-mlops.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "jerf8010"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "e1a4af0ea32606b9c9704ac2fb3fe38cc4f13f89"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [11]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/jerf8010/end-to-end-mlops.mlflow"
        )

        return model_evaluation_config

In [15]:
import joblib
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
import os
from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score
)
from urllib.parse import urlparse
from mlProject.utils.common import save_json


class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2
            }
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    

In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-18 11:55:57,982: INFO: common: yaml file config/config.yaml loaded successfully]
[2024-03-18 11:55:57,984: INFO: common: yaml file params.yaml loaded successfully]
[2024-03-18 11:55:57,985: INFO: common: yaml file schema.yaml loaded successfully]
[2024-03-18 11:55:57,986: INFO: common: created directory at: artifacts]
[2024-03-18 11:55:57,987: INFO: common: created directory at: artifacts/model_evaluation]
[2024-03-18 11:55:58,450: INFO: common: json file saved at: artifacts/model_evaluation/metrics.json]


/Users/emanuelfitta/Documents/learning/mlops_project/end-to-end-mlops/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/emanuelfitta/Documents/learning/mlops_project/end-to-end-mlops/.venv/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'ElasticModel' already exists. Creating a new version of this model...
2024/03/18 11:56:04 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to fi